في هذا التدريب العملي سنعمل على تضمين الكلمات مجددا. ولقد استعملنا التضمينات للنظر في تشابه الكلمات. لكننا الان سنستعمل التجميع بالمتوسطات k-means لتحويل تضمينات الكلمات إلى تصنيفات للكلمات، أي مجموعات من الكلمات يتوقع النموذج أن لها معان متشابهة.


ولنبدأ بتجهيز بيئة العمل.


In [1]:
from text_analytics import TextAnalytics
import os
import pandas as pd

ai = TextAnalytics()
ai.data_dir = os.path.join("..", "data")
print("Done!")

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Done!


سنبدأ برفع تضمينات الكلمات المدربة مسبقا. علما بأني أستعمل التضمينات المدربة على التغريدات، ويمكنك استعمال ما يأتي من التضمينات من خلال تغير اسم الملف:

     tweets: "sociolinguistics.english_all.gz"
     hotel reviews: "economic.hotel_reviews.gz"
     news articles: "economic.nyt.1931-2016.gz"
     speeches: "economic.congress.1931-2016.gz"
     19th century books: "stylistics.gutenberg_all.gz"

In [2]:
file = "economic.nyt.1931-2016.gz"
file = os.path.join(ai.data_dir, file)

#df = pd.read_csv(file)
#print(df)
#ai.train_word2vec(df)

ai.word_vectors = ai.deserialize("w2v_embedding", file + ".w2v_embedding.json")
ai.word_vectors_vocab = ai.deserialize("w2v_vocab", file + ".w2v_vocab.json")

#Build an index of each word
y = list(ai.word_vectors_vocab.keys())
    
print(ai.word_vectors)
print(list(ai.word_vectors_vocab.keys())[0:20])

[[-0.1235968   0.04748945  0.16285081 ...  0.03241363 -0.03300684
   0.26601192]
 [-0.0030584   0.04572405  0.10358463 ... -0.00173236 -0.10827369
   0.20938973]
 [-0.19914334  0.03242941  0.0766376  ...  0.12763536  0.02289162
   0.0425519 ]
 ...
 [ 0.05791736  0.24080583  0.11852352 ...  0.13066971  0.03170295
   0.20842037]
 [-0.05558461  0.00972249  0.02807915 ...  0.00696762  0.19038115
   0.08795328]
 [ 0.04808212  0.17134945 -0.06976075 ...  0.02019391  0.03982212
   0.0915439 ]]
['the_DET', 'of_ADP', 'a_DET', 'and_CCONJ', 'in_ADP', 'number_NOUN', 'to_PART', 'to_ADP', 'for_ADP', 'on_ADP', 'at_ADP', 'is_AUX', 'by_ADP', 'was_AUX', 'with_ADP', 'that_SCONJ', 'as_SCONJ', 'his_DET', 'from_ADP', 'it_PRON']


لنلق الآن نظرة على بعض المفردات. وتذكر أننا ولدنا عبارات باستعمال PMI، وأننا أضفنا أقسام الكلام بوصفها طريقة بسيطة للتمييز بين مختلف الكلمات حتى وإن كانت نفس السلسلة الحرفية.


In [3]:
import random

sample = random.sample(ai.word_vectors_vocab.keys(), 10)
print(sample)

['riskless_ADJ', 'leskopolkefuneralhomecom_PROPN', 'amts_NOUN', 'zarilla_PROPN', 'nyhistoryorg_PROPN', 'anodized_VERB', 'ketch_PROPN', 'jacq_PROPN', 'taubinlily_ADV', 'rushen_PROPN']


سنستعمل التجميع بالمتوسطات k-means الذي اعتمدنا عليه سابقا. والفرق الوحيد هو أننا هذه المرة نستعمله مع تضمين الكلمات!

متغيرنا *x* هو تضمينات الكلمات نفسها، وهو عبارة عن جدول من المتجهات لكل كلمة.

متغيرنا *y* هو صيغة الكلمة المقابلة لكل تضمين.

متغيرنا *k* هو عدد التجميعات المستهدفة، وهي 1000 هنا لأن لدينا عددا ضخما من الكلمات.

ولا نقارن ذلك بالوسم الفعلي، لذلك أسندنا القيمة false  إلى ari.


In [4]:
cluster_df = ai.cluster(x = ai.word_vectors, y = y, k = 1000)
print(cluster_df)

                   Label Cluster
0                the_DET     260
1                 of_ADP     242
2                  a_DET     834
3              and_CCONJ      96
4                 in_ADP     935
...                  ...     ...
281042   hierarchs_PROPN     822
281043     whiteonly_ADJ     217
281044  churubusco_PROPN      73
281045        amwest_ADJ     718
281046     citroens_NOUN     302

[281047 rows x 2 columns]


هذا كل شيء! الآن لدينا تجميع من الكلمات. ويعني هذا التجميع أن الآلة تتوقع أن الكلمات داخل التجميعة متشابهة بالاعتماد على توزيعها. فهو يتوقع أن لتلك الكلمات دلالات متشابهة. فهل هذا كذلك فعلا؟ لنلق نظرة من خلال طباعة بعض التجميعات العشوائية.

In [5]:
cluster = random.randint(0,999)
pd.set_option('display.max_rows', None)
select_df = cluster_df.loc[cluster_df["Cluster"] == cluster]
print(select_df)

                                 Label Cluster
1525                    democrat_PROPN     337
2587               representative_NOUN     337
4375                      senator_NOUN     337
5360                     speaker_PROPN     337
7614                 assemblyman_PROPN     337
8814                  congressman_NOUN     337
8848                    majority_PROPN     337
9602                      damato_PROPN     337
9730                   lieberman_PROPN     337
10866                   moynihan_PROPN     337
11202                   incumbent_NOUN     337
12249                  legislator_NOUN     337
13038                    tammany_PROPN     337
14005                    alfonse_PROPN     337
14696                    schumer_PROPN     337
15207                    democrat_NOUN     337
16692                 assemblyman_NOUN     337
16954                       vito_PROPN     337
17499                    ferraro_PROPN     337
18298                 torricelli_PROPN     337
19218        

في كل مرة تشغل هذا الكود البرمجي، ستظهر لك مجموعة مختلفة من الكلمات. ولذا لا أعرف ما الذي سيظهر لك تماما. ولكن ما يظهر عندي هو أن المجموعة تضم أسماء أماكن اجتماعية: معرض fair – كنيسة church – ملعب stadium – رواق gallery – استديو studio – كرنفال parade – منتجع resort – كازينو casino – سينما cinema. وتلك أماكن أو أحداث فعلية تؤدي وظيفة اجتماعية محددة هي الترفيه.

وهذا يدل على أن تضمين الكلمات، وقياس المسافة بينها، وخوارزمية التجميع يمكن أن تجرى معًا لتعرّفنا باللغة والعالم.
